In [37]:
import boto3
import os
import subprocess
import requests
import json

In [11]:
def s3_dir_ls(granule_dir):

    objs = []
    bucket = granule_dir.split("/")[2]
    key = "/".join(granule_dir.split("/")[3:])

    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(bucket)


    for obj in my_bucket.objects.filter(Prefix=key):
        objs.append("s3://" + bucket + "/" + obj.key)

    return objs[1:]

In [13]:
files = s3_dir_ls("s3://canopy-production-ml/full_congo_basin/02.17.21_CB_GEE_Pull/")

# POST Request

In [35]:
endpoint = " https://5oiulqb7cg.execute-api.us-east-1.amazonaws.com/default/cb-raster-process"
job_spec = { "granule_path": "s3://canopy-production-ml/full_congo_basin/02.17.21_CB_GEE_Pull/1000_full_congo_export_v12_all_bands_Feb_11_12_22_04_2021.tif", "dest_dir": "s3://canopy-production-ml/full_congo_basin/RGB/"}

response = requests.post(endpoint, json=job_spec)
print(response.text)

{"message":"Missing Authentication Token"}


In [34]:
!curl -X POST  https://5oiulqb7cg.execute-api.us-east-1.amazonaws.com/default -d sample

{"message":"Missing Authentication Token"}

# Functional Code

In [43]:
# define an empty list
paths = []

# open file and read the content in a list
with open('incomplete_granules_for_conversion.txt', 'r') as filehandle:
    for line in filehandle:
        # remove linebreak which is the last character of the string
        currentPlace = line[:-1]

        # add item to the list
        paths.append(currentPlace)

In [45]:
base_command = [
    'aws',
    'lambda',
    'invoke',
    '--function-name',
    'cb-raster-process',
    '--payload',
    'payload_dict_here',
    'response.json'
]

for file in paths:

    payload_dict = {'granule_path': file, 'dest_dir': r"s3://canopy-production-ml/full_congo_basin/RGB/"}
    test_command = base_command.copy()
    test_command[6] = json.dumps(payload_dict)
    subprocess.run(test_command, capture_output=True)